<a href="https://colab.research.google.com/github/Welle-Kim/Based-on-the-Review/blob/master/2_%5B%EB%AF%B8%EC%85%98%5D_%EC%9C%A0%EA%B0%80%EC%A0%95%EB%B3%B4_pandas_ipynb%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 코랩에서 한글 사용하기
- 한글 글꼴 설치 => [런타임]-[런타임 다시 시작] 클릭


In [ ]:
# 단계 1: 폰트 설치
import matplotlib.font_manager as fm

!apt-get -qq -y install fonts-nanum > /dev/null
fontpath = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'
font = fm.FontProperties(fname=fontpath, size=9)
fm._rebuild()

In [ ]:
# 단계 2: 런타임 재시작
import os
os.kill(os.getpid(), 9)

In [ ]:
# 단계 3: 한글 폰트 설정
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.font_manager as fm

# 마이너스 표시 문제
mpl.rcParams['axes.unicode_minus'] = False
	
# 한글 폰트 설정
path = '/usr/share/fonts/truetype/nanum/NanumGothicBold.ttf'
font_name = fm.FontProperties(fname=path, size=18).get_name()
plt.rc('font', family=font_name)
fm._rebuild()

### 서울시 유가정보(11개 구) 분서

#### [미션1] 서울시 유가정보 가져와 1개로 합치기
1. "/content" 폴더에 "data" 폴더 생성후 파일 업로드 
2. data 폴더에 있는 데이터 읽어와 oildf에 모두 저장
3. 데이터는 concat()을 이용해 데이터의 아래에 추가
4. encoding: cp949 또는 utf8이 같이 있어 에러 발생
        try: ~ except: ~ 로 해결



- data 폴더에 있는 파일명/하위폴더명 가져오기: os모듈
- for문을 이용 첫번째 파일부터 순차적으로 읽어와 oildf에 concat 진행
- 파일을 읽을시 encoding 에러
        try: 
            utf-8 형식 읽어오기
        except:
            cp949 형식 읽어오기
- 빈 DataFrame 변수 생성
        oildf=pd.DataFrame()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
import os

fnames=os.listdir("/content/data")

oildf = pd.DataFrame()

for fname in fnames:
    try:
        df0=pd.read_csv("/content/data/" + fname, header=2)
    except:
        df0=pd.read_csv("/content/data/" + fname, header=2, encoding="cp949")

    oildf=pd.concat([oildf, df0])  # 옵션: ignore_index=True : index 재설정

del df0

oildf.reset_index(drop=True, inplace=True)  # 

display(oildf.head(3))
oildf.info()

#### [미션2] 데이터 전처리
- NaN 값 포함된 데이터 전체 삭제
- "구" 필드 생성하기
- 데이터 출력 필드: 상호, 상표, 구, 세프여부, 휘발유, 경우 만 출력
- "휘발유", "경유": float64로 변경
- "휘발유", "경유" 데이터의 "-"를 평균값으로 수정


In [ ]:
#필드별 NAN 개수 확인
oildf.isna().sum()

In [ ]:
#구필드 생성하기
#loc[]함수를 이용해 주소에서 구 추출하기
gutext=[]
for i in oildf.index:
  gutext.append(oildf.loc[i,"주소"].split(" ")[1])
  

In [ ]:
#한줄쓰기로 표현
gutext=[oildf.loc[i,"주소"].split(" ")[1] for i in oildf.index]

In [ ]:
# iloc[] 함수를 이용해 주소에서 구 추출하기
gutxt=[oildf.iloc[i,2].split(" ")[1] for i in range(len(oildf))]

In [ ]:
print(gutxt)

In [ ]:
# "구" 필드 추가
oildf["구"]=gutxt

In [ ]:
oildf.shape

In [ ]:
#상호', '상표', "구", '셀프여부', '휘발유', '경유' 필드만 출력
# 필드명으로 DataFrame 생성하기
oildf1=oildf[["상호","상표","구", '셀프여부', '휘발유', '경유']]

In [ ]:
# iloc를 이용한 DataFrame 생성하기
oildf2=oildf.iloc[:,[1,3,-1,5,7,8]]
oildf2.dtypes

In [ ]:
display(oildf1.head(1))
display(oildf2.head(1))

In [ ]:
#NaN / "-" 값 0으로 변경하기
oildf2=oildf2.fillna(0)

In [ ]:
# 휘발유/경우 데이터에서 "-" 값만 찾아 0으로 변경
oildf2.loc[oildf2.휘발유=="-",'휘발유']=0
oildf2.loc[oildf2["경유"]=="-",'경유']=0

In [ ]:
# 고유값 확인하기
oildf2["경유"].unique()

In [ ]:
oildf2.head(3)

In [ ]:
# 휘발유/경우 실수형 자료로 변경하기
oildf2=oildf2.astype({"휘발유":float,"경유":float})
oildf2.dtypes 

In [ ]:
#휘발유/경유가 0인 값 평균으로 변경하기
oildf2.loc[oildf2.휘발유==0,"휘발유"]=oildf2.loc[oildf2.휘발유!=0,"휘발유"].mean()
oildf2.loc[oildf2.경유==0,"경유"]=oildf2.loc[oildf2["경유"]!=0,"경유"].mean()

In [ ]:
# 0인 값이 인는지 확인
oildf2.loc[oildf2["경유"]=="-",:]

In [ ]:
# oildf2에서 휘발유 최고가 검색
oildf2[oildf2["휘발유"]==oildf2["휘발유"].max()]

In [ ]:
# "구" 필드를 기준으로 평균 값 계산
guGroup=oildf2.groupby(by="구").mean()
guGroup.head()

In [ ]:
# 강북구 데이터만 출력
guGroup.loc[["강북구"]]

In [ ]:
# 강북구 / 양천구 출력
guGroup.loc[["강북구","양천구"]]

In [ ]:
guGroup.index

In [ ]:
# 강북구 ~ 서초구까지 의 휘발유값 출력
guGroup.loc["강북구":"서초구","휘발유"]

In [ ]:
# 구별 주유소 갯수
oildf2.groupby(["구"])["상표"].count()

In [ ]:
# 구별/상표별 주유소 갯수
oildf2.groupby(["구","상표"])[["휘발유","경유"]].count()

In [ ]:
# 구별/상표별 주유소 갯수 계산후 by 값을 데이터로 변경(reset_index() 실행)
oildf2.groupby(["구","상표"],as_index=False)["휘발유","경유"].count()

In [ ]:
# 그룹 안에서 원한는 데이터 확인
oildf2.groupby("구").get_group("노원구")

- DataFrame 데이터 저장
 * df.to_csv("경로/파일명.csv", index=None, encoding=None)
 * index: True(기본값/index값 저장), False(index 저장 생략)
 * encoding:


In [ ]:
# oildf2 csv 파일로 저장하기
oildf2.to_csv("서울시유류정보_0314.csv",index=False,encoding="cp949")

In [ ]:
# 코랩에서 저장한 파일 다운로드
from google.colab import files
files.download("서울시유류정보_0314.csv")